# Training CGNet

Purpose:
--------
The purpose of this notebook is to train CGnet for machine learning detection of atmospheric rivers and tropical cyclones.\
See ClimateNet repo here: https://github.com/andregraubner/ClimateNet

Authors/Contributors:
---------------------
* Teagan King
* John Truesdale
* Katie Dagon

## Import libraries

In [1]:
import os
import sys
import json
import numpy as np

sys.path.append("/glade/work/kdagon/ClimateNet") # append path to ClimateNet repo
from climatenet.utils.data import ClimateDatasetLabeled, ClimateDataset
from climatenet.models import CGNet
from climatenet.utils.utils import Config
from climatenet.track_events import track_events
from climatenet.analyze_events import analyze_events
from climatenet.visualize_events import visualize_events

from os import path

## Config file
Use `get_averages_and_standard_devs.ipynb` to calculate means/stds for given training dataset.

In [3]:
config = Config('/glade/work/kdagon/ML-extremes/trained_models/config_021523.json')

In [4]:
config.description

'The basic CGNet model. You can use this config to train your own model, or load it with our trained weights.'

In [18]:
config.train_batch_size

16

## Confirm GPU resources
Can request through JupyterHub launch page.\
Current resources request (2/15/23): 1 note, 4 cpu, 64GB mem, 2 V100 GPU

In [5]:
# requires loading pytorch into environment
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
2


## Instantiate CGNet model given config file

In [7]:
%%time
cgnet = CGNet(config)

CPU times: user 39 ms, sys: 1.44 ms, total: 40.5 ms
Wall time: 39.7 ms


In [19]:
cgnet.optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

## Set train, test data

In [10]:
train_path = "/glade/campaign/cgd/amp/jet/ClimateNet_12012020/portal.nersc.gov/project/ClimateNet/climatenet_new"

train = ClimateDatasetLabeled(path.join(train_path, 'train'), config)
test = ClimateDatasetLabeled(path.join(train_path, 'test'), config)

In [13]:
train.fields

{'TMQ': {'mean': 24.927238169017997, 'std': 15.817276954650879},
 'U850': {'mean': 1.0356735863118816, 'std': 8.29762077331543},
 'V850': {'mean': 0.20847854977498861, 'std': 6.231630802154541},
 'PSL': {'mean': 101095.03520124489, 'std': 1461.225830078125}}

In [16]:
train.length

398

## Train model

Memory use holding at ~13.6GB during training\
Each epoch takes ~1 min to run

In [11]:
cgnet.train(train) # use ~20 epochs for non-test
# IOU mean should be around 0.75 after all epochs?
# weights and measures site to look at ML performance
# maybe playing with those parameters could improve model

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1:


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch stats:
[[2.74805881e+08 2.24246630e+07 3.32901550e+07]
 [1.10457800e+06 3.46212000e+05 1.75557000e+05]
 [5.91861400e+06 1.16004800e+06 1.28992200e+07]]
IOUs:  [0.81413377 0.01373255 0.24136139] , mean:  0.35640923478750725
Epoch 2:


  0%|          | 0/25 [00:00<?, ?it/s]

...

Epoch stats:
[[3.18462197e+08 1.04817900e+06 1.10103230e+07]
 [5.30976000e+05 1.01621500e+06 7.91560000e+04]
 [5.35913400e+06 8.24270000e+04 1.45363210e+07]]
IOUs:  [0.94664674 0.36860077 0.46789687] , mean:  0.594381461820222
Epoch 19:


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch stats:
[[3.18753268e+08 1.05405600e+06 1.07133750e+07]
 [5.21263000e+05 1.02060300e+06 8.44810000e+04]
 [5.28368200e+06 7.75780000e+04 1.46166220e+07]]
IOUs:  [0.9477519  0.3700544  0.47493977] , mean:  0.5975820234985322
Epoch 20:


Loss: 0.7817890048027039: 100%|██████████| 25/25 [01:03<00:00,  2.55s/it]

Epoch stats:
[[3.18470937e+08 1.06732100e+06 1.09824410e+07]
 [5.12900000e+05 1.03284500e+06 8.06020000e+04]
 [5.20397100e+06 7.82790000e+04 1.46956320e+07]]
IOUs:  [0.94716048 0.37260633 0.47342764] , mean:  0.5977314837509967


Training is maxing out around 0.5977 IOU mean at last epoch (increase # epochs? play with batch size, other hyperparameters)

## Evaluate model on test data

In [22]:
test.fields

{'TMQ': {'mean': 24.927238169017997, 'std': 15.817276954650879},
 'U850': {'mean': 1.0356735863118816, 'std': 8.29762077331543},
 'V850': {'mean': 0.20847854977498861, 'std': 6.231630802154541},
 'PSL': {'mean': 101095.03520124489, 'std': 1461.225830078125}}

In [23]:
test.length

61

In [24]:
%%time
cgnet.evaluate(test)

100%|██████████| 4/4 [00:09<00:00,  2.46s/it]

Evaluation stats:
[[4.7718497e+07 1.6514600e+05 2.7706590e+06]
 [1.2726200e+05 1.6578600e+05 3.5460000e+03]
 [8.1544900e+05 1.3236000e+04 2.1893150e+06]]
IOUs:  [0.92483061 0.34904079 0.37797609] , mean:  0.5506158312616695
CPU times: user 1.62 s, sys: 1.69 s, total: 3.31 s
Wall time: 9.86 s


Testing data is at 0.55 IOU mean

## Save out model

In [26]:
# makes a folder by the name specified, puts the trained model and config file in there
cgnet.save_model('/glade/work/kdagon/ML-extremes/trained_models/trained_cgnet.021523')

In [48]:
# this how to load in previously trained model
cgnet.load_model('/glade/work/kdagon/ML-extremes/trained_models/trained_cgnet.021523')

## Set inference data

In [38]:
#year=2000
#inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/Climate_data_"+str(year)
inference_path = "/glade/scratch/tking/cgnet/historical_2000_2005/split_files" # torch input type error with these files

inference = ClimateDataset(inference_path, config)  # could test different config with std/means for inference data

In [39]:
inference.fields

{'TMQ': {'mean': 24.927238169017997, 'std': 15.817276954650879},
 'U850': {'mean': 1.0356735863118816, 'std': 8.29762077331543},
 'V850': {'mean': 0.20847854977498861, 'std': 6.231630802154541},
 'PSL': {'mean': 101095.03520124489, 'std': 1461.225830078125}}

In [40]:
inference.length

17520

## Inference mode

In [35]:
%%time
class_masks = cgnet.predict(inference) # masks with 1==TC, 2==AR

100%|██████████| 183/183 [06:38<00:00,  2.18s/it]


CPU times: user 2min 17s, sys: 1min 8s, total: 3min 26s
Wall time: 6min 46s


In [41]:
# with new files - need to reprocess TMQ as a float, not a double
class_masks = cgnet.predict(inference) # masks with 1==TC, 2==AR

  0%|          | 0/1095 [00:02<?, ?it/s]


RuntimeError: Input type (torch.cuda.DoubleTensor) and weight type (torch.cuda.FloatTensor) should be the same

## Save out masks

In [36]:
%%time
class_masks.to_netcdf("/glade/scratch/kdagon/cgnet/class_masks."+str(year)+".nc")

CPU times: user 55.9 ms, sys: 4.16 s, total: 4.22 s
Wall time: 7.53 s


## Track events
Create masks with event IDs

Note: memory spikes here - resource intensive!

In [ ]:
%%time
event_masks = track_events(class_masks)

In [ ]:
%%time
event_masks.to_netcdf("/glade/scratch/kdagon/cgnet/event_masks."+str(year)+".test.nc")

## Analyze events
Produce some visualizations

In [ ]:
analyze_events(event_masks, class_masks, "/glade/scratch/kdagon/cgnet/")

In [ ]:
visualize_events(event_masks, inference, "/glade/scratch/kdagon/cgnet/")   

## Weights and Biases
https://docs.wandb.ai/quickstart \
Still figuring out how this works...

In [42]:
import wandb

In [44]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········


wandb: Appending key for api.wandb.ai to your netrc file: /glade/u/home/kdagon/.netrc


True

In [46]:
wandb.init(project="climatenet-test", entity="katie-dagon")

Ignoring settings passed to wandb.setup() which has already been configured.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········


wandb: Appending key for api.wandb.ai to your netrc file: /glade/u/home/kdagon/.netrc
wandb: wandb version 0.13.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20230215_172859-2auiboc1
wandb: Syncing run decent-river-1


In [47]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 20,
  "batch_size": 16
}

In [49]:
wandb.log({"loss": loss})

NameError: name 'loss' is not defined